# Taller Grupal 1 - Métodos y aplicaciones de analítica I

**Anderson Alvarado & Juan Felipe Gonzalez Rodriguez**

# Librerias

# Carga de Datos

# Preguntas

## a) Están interesados en saber qué productos o categorías de productos se venden bien juntos.

## b) A partir de ello, desean recomendaciones específicas separadas para su tienda virtual y sus tiendas físicas.

## c) Finalmente, están curiosos acerca de qué otra información podría darles las diversas bases de datos allí presentes que pudiese ser valiosa para comprender las dinámicas del comportamiento de sus usuarios /productos.